<a href="https://colab.research.google.com/github/sakuna47/Stock_Price_Prediction-/blob/Frontend/SPP_Frontend_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import pickle
import numpy as np
import yfinance as yf
import pandas as pd

# Load the model and scaler
with open("stock_model.pkl", "rb") as f:
    model = pickle.load(f)

with open("scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

# Streamlit app
st.title("Stock Price Prediction Web App")

# Input field for stock ticker
ticker = st.text_input("Enter Stock Ticker (e.g., AAPL):")

# Predict button
if st.button("Predict"):
    if ticker:
        try:
            # Download historical stock data
            data = yf.download(ticker, start="2010-01-01", end="2025-02-15")

            if not data.empty:
                # Print column names for debugging
                #st.write("Downloaded Data Columns:", list(data.columns))

                # Fill missing values
                data = data.fillna(method="ffill")

                # Expected features in the dataset
                expected_features = ["Open", "High", "Low", "Volume"]

                # Get available features
                available_features = [col for col in expected_features if col in data.columns]

                # Check if we have all required features
                if len(available_features) < len(expected_features):
                    missing_features = list(set(expected_features) - set(available_features))
                    st.error(f"Missing features: {missing_features}. Cannot make predictions.")
                else:
                    # Select available features
                    X = data[available_features]

                    # Scale the features
                    X_scaled = scaler.transform(X)

                    # Make prediction for the next day's closing price
                    prediction = model.predict(X_scaled)

                    st.success(f"Predicted Next Closing Price: ${prediction[-1]:.2f}")
            else:
                st.error("Invalid stock ticker or no data available.")

        except Exception as e:
            st.error(f"An error occurred: {str(e)}")

    else:
        st.warning("Please enter a stock ticker.")
